In [17]:
import asyncio
import json
import websockets
import base64
import sounddevice as sd
import numpy as np
import nest_asyncio

API_KEY = "XdnFgR8ESRCNfcoh6V5dfKBZaQD6e68jauRrfUVvl3OYf2RGfyOZ3q7oan7uhB6J"
EVI_URL = f"wss://api.hume.ai/v0/evi/chat?access_token={API_KEY}"

session_settings = {
    "type": "session_settings",
    "voice": {
        "provider": "custom",
        "custom_voice": {
            "name": "custom_kora",
            "base_voice": "KORA",
            "parameter_model": "20240715-4parameter",
            "speech_rate_multiplier": 1.5,
            "parameters": {
                "gender": -50,
                "huskiness": 0,
                "nasalness": 0,
                "pitch": 0
            }
        }
    },
    "audio": {
        "encoding": "linear16",
        "sample_rate": 24000,
        "channels": 1
    }
}

def play_audio(audio_data):
    # Convert audio data to numpy array
    audio_np = np.frombuffer(audio_data, dtype=np.int16)

    # Play the audio
    sd.play(audio_np, samplerate=24000)
    sd.wait()

async def chat_with_evi():
    async with websockets.connect(EVI_URL) as websocket:
        # Send SessionSettings
        await websocket.send(json.dumps(session_settings))
        print("SessionSettings sent")

        while True:
            # Send user input
            user_input = input("You: ")
            if user_input.lower() == 'quit':
                break
            await websocket.send(json.dumps({"type": "user_input", "text": user_input}))

            # Receive and process EVI's response
            while True:
                response = await websocket.recv()
                response_json = json.loads(response)

                if response_json['type'] == 'audio_output':
                    audio_data = base64.b64decode(response_json['data'])
                    play_audio(audio_data)
                elif response_json['type'] == 'assistant_message':
                    print("EVI:", response_json['message']['content'])
                elif response_json['type'] == 'assistant_end':
                    break
                else:
                    print("Received:", response_json)

nest_asyncio.apply()
await chat_with_evi()

IndentationError: expected an indented block after 'while' statement on line 83 (1186907059.py, line 86)